In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
from torchvision import models, transforms

# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s]


In [2]:
# Method for training and evaluating the model
def train_and_evaluate(model, train_loader, validation_loader, n_epochs=10, lr=0.001, device=None):
    if device is None:
        # setting up CUDA if available, otherwise, CPU
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)  # transferring the model to the selected device (GPU or CPU)

    # Initializing the loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)  # model.parameters() is a built-in method provided by PyTorch's nn.Module that collects all the learnable parameters of our model

    # Training loop
    for epoch in range(n_epochs):
        model.train()  # Setting model to training mode
        training_loss = 0.0
        # tracking number of correctly predicted instances
        correct_train = 0
        # tracking total number of instances
        total_train = 0

        # Iterate over the training data
        for images, labels in train_loader:
            # transferring images and labels to the current device (GPU or CPU)
            images, labels = images.to(device), labels.to(device)
            # clearing the gradients of all optimized variables
            optimizer.zero_grad()
            # Forward prop: computing predicted outputs by passing inputs to the model
            outputs = model(images)
            # calculating the loss between predicted and true labels
            loss = loss_function(outputs, labels)
            # Backward prop: computing gradient of the loss with respect to parameters
            loss.backward()
            # optimization step (parameter update)
            optimizer.step()
            # updating running training loss
            training_loss += loss.item() * images.size(0)
            # converting output probabilities to predicted class
            _, predicted_train = torch.max(outputs.data, 1)
            # updating total number of instances
            total_train += labels.size(0)
            # updating correctly predicted instances
            correct_train += (predicted_train == labels).sum().item()

        # average loss over an epoch
        training_loss /= len(train_loader.sampler)
        # train accuracy
        train_accuracy = correct_train / total_train

        # Evaluation phase
        model.eval()  # Setting model to evaluation mode
        # initializing the validation loss for the epoch
        validation_loss = 0.0
        # tracking number of correctly predicted instances
        correct_val = 0
        # tracking total number of instances
        total_val = 0
        # disable gradient calculation for validation, saving memory and computations
        with torch.no_grad():
            # iterating over the validation data loader
            for images, labels in validation_loader:
                # transferring images and labels to the current device (GPU or CPU)
                images, labels = images.to(device), labels.to(device)
                # Forward prop: computing predicted outputs by passing inputs to the model
                outputs = model(images)
                # calculating the loss between predicted and true labels
                loss = loss_function(outputs, labels)
                # updating running validation loss
                validation_loss += loss.item() * images.size(0)
                # converting output probabilities to predicted class
                _, predicted = torch.max(outputs.data, 1)
                # updating total number of instances
                total_val += labels.size(0)
                # updating correctly predicted instances
                correct_val += (predicted == labels).sum().item()

        # average validation loss over an epoch
        validation_loss /= len(validation_loader.sampler)
        # validation accuracy
        val_accuracy = correct_val / total_val

        # printing training and validation results
        print(f'Epoch {epoch+1}, Training Loss: {training_loss:.4f}, Training Accuracy: {train_accuracy*100:.2f}, \nValidation Loss: {validation_loss:.4f}, Validation Accuracy: {val_accuracy*100:.2f}')
        wandb.log({"train_accuracy": train_accuracy * 100, "training_loss": training_loss})
        wandb.log({"val_accuracy": val_accuracy * 100, "val_loss": validation_loss})

In [3]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

key = input('Enter your API:')
wandb.login(key=key)  #25c2257eaf6c22aa056893db14da4ee2bf0a531a

Enter your API: 25c2257eaf6c22aa056893db14da4ee2bf0a531a


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
sweep_config = {
    'method': 'bayes',
    'name' : 'B2_kaggle',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'freeze': {
            'values': [0.7,0.8,0.9] 
        },
        'epochs': {
            'values': [5, 7]
        },   
        'batch_size': {
            'values': [32, 64]
        },
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="Deep_learning_A2_partB")

Create sweep with ID: 40m9jaz2
Sweep URL: https://wandb.ai/parthasakhapaul/Deep_learning_A2_partB/sweeps/40m9jaz2


In [ ]:
def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        run_name = "-freeze-"+str(wandb.config.freeze)+"-epochs-"+str(wandb.config.epochs)+"-batch_size-"+str(wandb.config.batch_size)  
        wandb.run.name=run_name
        
        
        def prepare_data_loaders(data_augment, batch_size=64):
            if data_augment:
                # With data augmentation
                train_transform = transforms.Compose([
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.450, 0.470, 0.490], std=[0.200, 0.220, 0.240]),
                ])
            else:
                # Without data augmentation
                train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                ])

            # Load datasets
            train_data = datasets.ImageFolder(root='/kaggle/input/inaturalist-12k/inaturalist_12K/train', transform=train_transform)
        #     validation_dataset = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=val_transform)
        #     print(len(train_data))
            # print(train_data[9998])
            # Creating data indices for training and validation splits:
            dataset_size = len(train_data)  #9999
            indices = list(range(dataset_size)) #[0,1,...,9998]
            split = int(np.floor(0.2 * dataset_size))  #1999
        #     print(split)
            np.random.shuffle(indices)
            train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
            # print((val_indices))

            # Creating data samplers and loaders
            train_sampler = SubsetRandomSampler(train_indices)
            valid_sampler = SubsetRandomSampler(val_indices)
            # print(type(train_sampler))

            train_loader = DataLoader(train_data, batch_size=64, sampler=train_sampler)
            # print(len(train_loader))
            validation_loader = DataLoader(train_data, batch_size=64, sampler=valid_sampler)

            return train_loader, validation_loader


        # Load pre-trained ResNet50
        model = models.resnet50(pretrained=True)

        # In PyTorch, freezing a layer means setting its requires_grad attribute to False
        for param in model.parameters():
            param.requires_grad = False

        # Replace the last layer
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)  # since 10 classes in the iNaturalist dataset

        # transformations
        data_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        print(model)
        

        # Calculate total parameters and target
        total_params = sum(p.numel() for p in model.parameters())
        target_freeze = total_params * wandb.config.freeze

        # function to freeze parameters
        def freeze_parameters(model, target_freeze):
            frozen_params = 0
            for param in model.parameters():
                if frozen_params >= target_freeze:
                    break  # Stop freezing parameters when target is achieved
                param.requires_grad = False
                frozen_params += param.numel()  # tracking of how many parameters have been frozen

        # Freezing of parameters
        freeze_parameters(model, target_freeze)
        # print(model)

        # Replacing the last layer
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)  # since 10 classes in the iNaturalist dataset
        
        # preparing data loaders
        train_loader, validation_loader = prepare_data_loaders('false',batch_size = wandb.config.batch_size)
        # Model training
        train_and_evaluate(model, train_loader, validation_loader, n_epochs=wandb.config.epochs, lr=0.001)        

wandb.agent(sweep_id, function=main, count=30)
wandb.finish()

wandb: Agent Starting Run: 855xzqyu with config:
wandb: 	batch_size: 32
wandb: 	epochs: 7
wandb: 	freeze: 0.7
wandb: Currently logged in as: parthasakhapaul. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

train_accuracy,▁▇▇██▇█
training_loss,█▂▂▁▁▁▁
val_accuracy,▁▅▇▆█▆▇
val_loss,█▄▁▂▁▂▂
train_accuracy,76.425
training_loss,0.71129
val_accuracy,75.03752
val_loss,0.75709


wandb: Agent Starting Run: 1p1fducb with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	freeze: 0.8


...

wandb: Agent Starting Run: tu31xa8k with config:
wandb: 	batch_size: 64
wandb: 	epochs: 7
wandb: 	freeze: 0.8


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

train_accuracy,▁▇▇▇███
training_loss,█▃▂▂▁▁▁
val_accuracy,▁▂▅▄▇█▆
val_loss,█▄▄▂▂▁▄
train_accuracy,76.3
training_loss,0.70449
val_accuracy,76.43822
val_loss,0.76728


wandb: Agent Starting Run: 4fpw3of2 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 7
wandb: 	freeze: 0.8


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 